<a href="https://colab.research.google.com/github/phisan-chula/2021-LDP_Design/blob/main/LDP_Mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Design of a Low Distortion Projection for a Mini Project**</br>  
LDP_Inspection : program to design  low distortion projections (LDPs) using conformal map projections for minimizing linear distortion between projected coordinates eg. UTM grid and the true distance at the surface of the engineering project.</br>  

Phisan Santitamonont,</br>  
Faculty of Engineering, Chulalongkorn University © 2022</br>
*Phisan.Chula@gmail.com*

In [8]:
%%capture
! pip install pygeodesy
! pip install pyproj
! pip install requests
! pip install folium

In [9]:
from google.colab import drive
drive.mount('/content/drive')
import requests
import pygeodesy as pgd
import numpy as np
from shapely.geometry import Point
import pandas as pd
import pyproj
# download tgm2017 geoid model to you own storage
TGM17 = r'/content/drive/MyDrive/Geodesy_Cache/tgm2017-1.pgm'

####################################################
#  Projection Plane (PP) @ lat,lng and topo(SRTM/MSL) height
#POS, HOR_BUF,  VER_BUF =[13.5411528, 99.8240430],2_000,20.0 # Rachburi,
POS, HOR_BUF,  VER_BUF =[14.519354, 101.017051],1_000,50  # CU-SBR
####################################################

res = requests.get( r'https://api.opentopodata.org/v1/srtm30m?locations={},{}'.format( *POS ) )
MSL = res.json()['results'][0]['elevation']
ELLPS  = pgd.datums.Ellipsoids.WGS84
GEOID = pgd.geoids.GeoidKarney( TGM17 )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
def dd2DMS( dd, PREC=7, POS=''  ):
    '''convert degree to DMS string'''
    return pgd.dms.toDMS( dd, prec=PREC,pos=POS )
def CalcLDP( row ):
    UNDUL = GEOID.height( row.lat,row.lng )
    RG   = ELLPS.rocGauss( row.lat )
    h     = UNDUL + row.MSL
    HSF = RG/(RG+h)
    FACTOR = pyproj.Proj( LDP).get_factors( row.lng, row.lat )
    PSF = FACTOR.meridional_scale
    CSF = PSF*HSF
    CSF_ppm = (CSF-1)*1E6
    TR = pyproj.Transformer.from_crs( 'epsg:4326', LDP )
    LDP_E,LDP_N = TR.transform( row.lat, row.lng )
    return [UNDUL, h, HSF, PSF, CSF, CSF_ppm, LDP_E, LDP_N]

In [11]:
res = requests.get( r'https://api.opentopodata.org/v1/srtm30m?locations={},{}'.format( *POS ) )
MSL = res.json()['results'][0]['elevation']
####################################################
ELLPS  = pgd.datums.Ellipsoids.WGS84
GEOID = pgd.geoids.GeoidKarney( TGM17 )
UNDUL = GEOID.height( *POS )
HAE = UNDUL + MSL
RG = ELLPS.rocGauss( POS[0] )
k0 = np.round(1 + HAE/RG, 6)  #  M.Dennis 2016 : Ground Truth ... (...5 to 6 digits)
print( f'Projection Plane : lat = {dd2DMS(POS[0]):}  lng = {dd2DMS(POS[1]):}  ')
print( f'MSL = {MSL:.1f} m. HAE={HAE:.1f} m. ,  N={UNDUL:.1f} m.  ==> k0 = {k0:.6f}' )

Projection Plane : lat = 14°31′09.6744″  lng = 101°01′01.3836″  
MSL = 29.0 m. HAE=-0.7 m. ,  N=-29.7 m.  ==> k0 = 1.000000


In [12]:
COL_LDP = ['UNDUL', 'h','HSF','PSF','CSF', 'CSF_ppm', 'LDP_E', 'LDP_N']
FLT_RND = {'UNDUL':2, 'h':2, 'CSF_ppm':1 , 'LDP_E':3, 'LDP_N':3  }
TM = r'+proj=tmerc +lat_0=0.0 +lon_0={} +k_0={}  +x_0={}  +y_0={}  +a={} +b={} +units=m +no_defs'
####################### LDP Design ############################
#LDP = pyproj.CRS( TM.format( 99+49/60, k0, +2500, -1490000, ELLPS.a, ELLPS.b ) )  # RACHBURI fine tune !!!
LDP = pyproj.CRS( TM.format( 101+1/60, k0, +2500, -1490000, ELLPS.a, ELLPS.b ) )  # RACHBURI fine tune !!!
############################################################
# create 3 projection planes and test them
EWNS =Point( POS[1],POS[0] ).buffer( HOR_BUF/111_000 , cap_style = 3  ).exterior.coords.xy
PP_EWNS = np.vstack( (np.array( [POS[1],POS[0]] ) , np.array(EWNS).T) )[:-1]
dfPP = pd.DataFrame( {'Point':['P0','P1','P2','P3','P4'], 'lng':PP_EWNS[:,0], 'lat':PP_EWNS[:,1]  } )
dfPP = pd.concat( 3*[dfPP] ,  ignore_index=True) # create 3 planes ...
dfPP['MSL'] = 5*[MSL+VER_BUF]+  5*[MSL] +  5*[MSL-VER_BUF]
dfPP[COL_LDP] = dfPP.apply( CalcLDP, axis=1, result_type='expand')
print( LDP )
dfPP.round( FLT_RND )

+proj=tmerc +lat_0=0.0 +lon_0=101.01666666666667 +k_0=1.0  +x_0=2500  +y_0=-1490000  +a=6378137.0 +b=6356752.314245179 +units=m +no_defs +type=crs


,Point,lng,lat,MSL,UNDUL,h,HSF,PSF,CSF,CSF_ppm,LDP_E,LDP_N
0,P0,101.017051,14.519354,79.0,-29.72,49.28,0.999992,1.0,0.999992,-7.7,2541.426,115807.846
1,P1,101.026060,14.528363,79.0,-29.68,49.32,0.999992,1.0,0.999992,-7.7,3512.439,116804.661
2,P2,101.026060,14.510345,79.0,-29.68,49.32,0.999992,1.0,0.999992,-7.7,3512.521,114811.073
3,P3,101.008042,14.510345,79.0,-29.76,49.24,0.999992,1.0,0.999992,-7.7,1570.335,114811.070
4,P4,101.008042,14.528363,79.0,-29.77,49.23,0.999992,1.0,0.999992,-7.7,1570.410,116804.657
5,P0,101.017051,14.519354,29.0,-29.72,-0.72,1.000000,1.0,1.000000,0.1,2541.426,115807.846
6,P1,101.026060,14.528363,29.0,-29.68,-0.68,1.000000,1.0,1.000000,0.1,3512.439,116804.661
7,P2,101.026060,14.510345,29.0,-29.68,-0.68,1.000000,1.0,1.000000,0.1,3512.521,114811.073
8,P3,101.008042,14.510345,29.0,-29.76,-0.76,1.000000,1.0,1.000000,0.1,1570.335,114811.070
9,P4,101.008042,14.528363,29.0,-29.77,-0.77,1.000000,1.0,1.000000,0.1,1570.410,116804.657


**User will input project's control points, RTKs, setting-out coordinates here...**

In [13]:
from io import StringIO
CSV = '''Point,lat,lng,MSL
RTK1,13.542153,99.824043,20.0
RTK2,13.540153,99.821043,-30.0
RTK3,13.541153,99.822043,2.0
'''
CALC_RTK = False
if CALC_RTK:
    dfRTK = pd.read_csv( StringIO(CSV) )
    dfRTK[COL_LDP] = dfRTK.apply( CalcLDP, axis=1, result_type='expand')
    dfRTK = dfRTK.round( FLT_RND )
    dfRTK

In [14]:
import folium
from folium.features import DivIcon

icon=folium.Icon(color='red', icon='plus' )
map = folium.Map(location =[POS[0],POS[1]],  zoom_start = 14 )
for grp,row in dfPP.groupby( [ 'Point' ] ):
    csf = list( row.CSF_ppm.round(1) )
    location = (row.iloc[0].lat , row.iloc[0].lng)
    folium.CircleMarker(location=location, radius=15, color='red', fill_color ='red', fill_opacity=0.5,
                                tooltip=f'<b>CSF:{csf:}</b>').add_to(map)
    folium.map.Marker(location, icon=DivIcon( icon_size=(30,30), icon_anchor=(5,14),
                          html=f'<div style="font-size: 14pt">%s</div>' % str(grp) ) ).add_to(map)
if CALC_RTK:
    for i,row in dfRTK.iterrows():
        location = (row.lat,row.lng)
        folium.CircleMarker(location=location, tooltip=f'<b>CSF:{row.CSF_ppm:}</b>',
                            radius=15, color='green', fill_color ='green', fill_opacity=0.5 ).add_to(map)
        folium.map.Marker(location, icon=DivIcon( icon_size=(30,30), icon_anchor=(5,14),
                              html=f'<div style="font-size: 14pt">%s</div>' % row['Point'], ) ).add_to(map)
map